In [96]:
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [117]:
dataset = 'cityscapes'
baseline_stats_cs = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/paper_logs/baseline_hs_100.pkl', 'rb'))
stats_cs = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/paper_logs/stats_with_h_99.pkl', 'rb'))
dataset = 'acdc'
baseline_stats_acdc = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/paper_logs/baseline_hs_100.pkl', 'rb'))
stats_acdc = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/paper_logs/stats_with_h_0.pkl', 'rb'))

In [118]:
# getting biggest difference in certified info gain between SegCertify and AdaptiveCertify
prev_diff = -1
h = [0, 1, 2, 3, 4]
for im_name in stats_cs.keys():
    im_stats = stats_cs[im_name]
    for n_h in im_stats.keys():
        n, _ = n_h
        

        c_ig_segcertify = stats_cs[im_name][(n, 0)]['c_info_gain']/stats_cs[im_name][(n, 0)]['num_pixels']
        c_ig_adaptive = stats_cs[im_name][(n, 4)]['c_info_gain']/stats_cs[im_name][(n, 4)]['num_pixels']
        diff = c_ig_adaptive - c_ig_segcertify

        if diff > prev_diff:
            prev_diff = diff
            cherry_im = im_name
            cherry_n = n
            #print(cherry_im, diff)

In [99]:
print(f'Biggest different in certified info. gain between SegCertify and AdaptiveCertify: (image={cherry_im} @ n={cherry_n}, diff={prev_diff})')


Biggest different in certified info. gain between SegCertify and AdaptiveCertify: (image=frankfurt_000001_014406 @ n=100, diff=0.07252046885225072)


In [128]:
### adaptive n0, constant thresholds
n0_adaptive = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/cityscapes_logs/paper_logs/adaptive_n0_and_thresholds/3.pkl', 'rb'))
# certified_info_gain vs. n (for AdaptiveCertify)

graph_dict = {} # should have (n, n0, i) as keys, values: sum
for name, name_dict in n0_adaptive.items():
    for k in name_dict.keys():
        # only graph_adaptive
        if k not in graph_dict:
            graph_dict[k] = {'num_pixels':0, 
                                      'certified_pixels':0, 
                                      'c_info_gain':0, 
                                      'certified_pos':0, 
                                      'boundary_pixels':0, 
                                      'nonboundary_pixels':0, 
                                      'pos_certified_boundary_pixels':0, 
                                      'pos_certified_nonboundary_pixels':0,
                                      'num_h_pixels':{}, 'pos_h_pixels':{},
                                      'ig_per_class_dict': {i:{'ig': np.zeros(19), 'pixels_count': np.zeros(19)} for i in range(4)}}
            graph_dict[k]['num_pixels'] += name_dict[k]['num_pixels']
            graph_dict[k]['certified_pixels'] += name_dict[k]['certified_pixels']
            graph_dict[k]['certified_pos'] += name_dict[k]['certified_pos']
            graph_dict[k]['c_info_gain'] += name_dict[k]['c_info_gain']
            for i in name_dict[k]['ig_per_class_dict'].keys():
                graph_dict[k]['ig_per_class_dict'][i]['ig'] += name_dict[k]['ig_per_class_dict'][i]['ig']
                graph_dict[k]['ig_per_class_dict'][i]['pixels_count'] += name_dict[k]['ig_per_class_dict'][i]['pixels_count']

In [129]:
# Adaptive Certify
# make adaptive n0 graph
f_choice = 'exp_0.40'
x_axis_title = 'Samples budget (N)'
y_axis_title = 'Certified Info. Gain'
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
ns = sorted(list(set(np.array(list(graph_dict.keys()))[:, 0])))
n0s = list(set(np.array(list(graph_dict.keys()))[:, 1]))
hs = list(set(np.array(list(graph_dict.keys()))[:, 2]))
h = 0 # only adaptive
fig = make_subplots(rows=3, cols=2, shared_yaxes=True, subplot_titles=("SegCertify", 'AdaptiveCertify'))

show_legend = True
for row_i, h in enumerate([0, 4]):
    for col_i, g in enumerate(['certified_pixels', 'certified_pos', 'c_info_gain']):
        for idx, n0 in enumerate(n0s):
            y_axis = []
            x_axis = []
            for n in ns:
                if h==0: f=None
                else: f=f_choice
                mean_g = graph_dict[(n, n0, f, h)][g]/graph_dict[(n, n0, f, h)]['num_pixels']
                if g == 'c_info_gain': mean_g /= np.log(19)
                y_axis.append(mean_g)
                x_axis.append(n)
            if n0 < 1: n0 = n0*100; ext = '%'
            else: ext = ''
            name = f'n0={n0}'+ext
            

            fig.add_scatter(x=x_axis, y=y_axis, # text=y_axis, 
                            col=row_i+1, row=col_i+1, 
                            name=name, 
                            line_color=colors[idx], 
                            line = dict( width=2), 
                            showlegend=show_legend, mode="lines+markers+text", textposition="top center" )


        fig.update_yaxes(title_text= g, col=row_i+1, row=col_i+1)
        show_legend = False
    if col_i == 2: fig.update_xaxes(title_text=x_axis_title, type='category', col=row_i+1, row=col_i+1)
fig.update_layout(
margin=dict(l=20, r=20, t=20, b=20),
)
fig.show()


In [130]:
threshold_functions = {'linear': [0.25* i for i in range(1, 4)],
'exp_0.75': [0.75**(4-i) for i in range(1, 4)],
'exp_0.40': [0.4**(4-i) for i in range(1, 4)],
'linear_n': [(100/300*0.25)*i for i in range(1, 4)],
'exp_n': [(100/300*0.25)**(4-i) for i in range(1, 4)]}


fig = go.Figure()
for k, y_axis in threshold_functions.items():
    fig.add_scatter(x=list([str(x) for x in range(3, 0, -1)]),  y=y_axis, name=f'{k}')
fig.update_xaxes(title_text='Hierarchy cuttoff (h_x if diff < y)', type='category')
fig.show()


In [131]:
def linear_n(n):
    return [(n/300*0.25)*i for i in range(1, 4)]
def exp_n(n):
    return [(n/300*0.75)**(4-i) for i in range(1, 4)]
fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=("Linear in n", 'Exponential in n'))

for n in range(100, 301, 50):
    fig.add_scatter(x=list([str(x) for x in range(4, 1, -1)]),  y=linear_n(n), name=f'linear_{n}', row=1, col=1)
for n in range(100, 301, 50):
    fig.add_scatter(x=list([str(x) for x in range(4, 1, -1)]),  y=exp_n(n), name=f'exp_{n}', row=1, col=2)
fig.show()

In [132]:
#  Graph different threshold functions
# choice: 
n0_choice = 10

x_axis_title = 'Samples budget (N)'
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
ns = sorted(list(set(np.array(list(graph_dict.keys()))[:, 0])))
n0s = list(set(np.array(list(graph_dict.keys()))[:, 1]))
fs = list(set(np.array(list(graph_dict.keys()))[:, 2]))

fig = make_subplots(rows=1, cols=3, shared_yaxes=True, subplot_titles=("Mean Percentage of Certified Pixels", 'Mean Certified Accuracy', 'Mean Certified Info. Gain'))
show_legend = True
row_i = 0
for col_i, g in enumerate(['certified_pixels', 'certified_pos', 'c_info_gain']):
    for idx, f in enumerate(fs):
        if f is None: h = 0; name = 'SegCertify'
        else: h=4;  name=f
        y_axis = []
        x_axis = []
        for n in ns:
            mean_g = graph_dict[(n, n0_choice, f, h)][g]/graph_dict[(n, n0_choice, f,  h)]['num_pixels']
            if g == 'c_info_gain': mean_g /= np.log(19)
            y_axis.append(mean_g)
            x_axis.append(n)
        

        fig.add_scatter(x=x_axis, y=y_axis, # text=y_axis, 
                        row=row_i+1, col=col_i+1, 
                        name=name, 
                        line_color=colors[idx] if h == 4 else 'black', 
                        line = dict( width=2), 
                        showlegend=show_legend, mode="lines+markers+text", textposition="top center" )


    fig.update_yaxes(title_text= g, row=row_i+1, col=col_i+1)
    show_legend = False
    fig.update_xaxes(title_text=x_axis_title, type='category', row=row_i+1, col=col_i+1)

# add SegCertify using n0=0.2

fig.update_layout(
margin=dict(l=20, r=20, t=20, b=20),
)
fig.show()

In [133]:
labels_dict_h1 = pickle.load(open('/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/data/cityscapes_trainIds.pkl', 'rb')) # 19
labels_dict_h1[0] = 'drivable'
labels_dict_h2 = {0:'drivable', 1:'sky', 2:'construction & vegetation', 3:'traffic sign', 4:'human', 5:'vehicle', 6:'flat obstacle', 7:'abstain'} # 7
labels_dict_h3 = {0: 'drivable', 1:'static obstacle', 2: 'dynamic obstacle', 3:'flat obstacle', 4: 'abstain'} # 5
labels_dict_h4 = {0: 'drivable', 1:'obstacle', 2:'abstain'} # 3
n0_adaptive = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/cityscapes_logs/paper_logs/adaptive_n0_and_thresholds/0.pkl', 'rb'))
labels_dicts = [labels_dict_h1, labels_dict_h2, labels_dict_h3, labels_dict_h4]


In [138]:

ig_all = {}
for h in [0, 4]:
    if h==0:
        k = (300, 20, None, h)
    else:
        k = (300, 20, 'exp_0.40', 4)
    ig_all[h] = {}
    if h==0: H=1
    else: H=4
    for i in range(H):
        for j in range(len(labels_dicts[i].keys())-1):
            label_name = labels_dicts[i][j] 
            # if label_name == 'drivable': h_name='H0'; 
            h_name = f'H{i}'
            if graph_dict[k]['ig_per_class_dict'][i]['pixels_count'][j] == 0:
                ig_all[h][f'{label_name}_{h_name}'] = 0
            else:
                ig_all[h][f'{label_name}_{h_name}'] = graph_dict[k]['ig_per_class_dict'][i]['ig'][j]/graph_dict[k]['ig_per_class_dict'][i]['pixels_count'][j]/np.log(19)


In [137]:
(300, 20, 'exp_0.4', 4)

f_choice = 'exp_0.40'
x_axis_title = 'Class and hierarchy'
y_axis_title = 'Mean Certified Info. Gain'
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
subplot_titles = ("SegCertify", 'AdaptiveCertify')
ig_dict_seg = dict(sorted(ig_all[0].items(), key=lambda x:x[1], reverse=True))
ig_dict_adaptive = dict(sorted(ig_all[4].items(), key=lambda x:x[1], reverse=True))

fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=subplot_titles)

show_legend = True
up_to = 15
for row_i, h in enumerate([0, 4]):
    if h==0: d = ig_dict_seg
    else: d = ig_dict_adaptive
    fig.add_bar(x=list(d.keys())[:up_to], y=list(d.values())[:up_to], # text=y_axis, 
                    col=row_i+1, row=1, name=subplot_titles[row_i])

    show_legend = False
fig.update_layout(
margin=dict(l=20, r=20, t=20, b=20),
)
fig.show()

$acc(B_m) = \frac{1}{|B_m|} \sum_{i \in B_m} \mathbf{1}(\hat{y}_i = y_i)$

$conf(B_m) = \frac{1}{|B_m|} \sum_{i \in B_m} \hat{p}_i$

The Expected Calibration Error (ECE) is taking the weighted average of the bins' accuracy/cofidence differences.

ECE=∑Mm=1|Bm|n|acc(Bm)−conf(Bm)|

The Maximum Calibration Error (MCE) focuses more on high risk applications where the maximum accuracy/confidence difference is more important than just the average.

MCE=maxm|acc(Bm)−conf(Bm)|